In [1]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime, date, time
#import pyodbc
import requests
from tqdm import tqdm

In [2]:
CSV_filename = 'CC_export__executionsteplog__202309201735.csv'
CSV_filename = '_SELECT_ExecutionLog_scriptname_ExecutionStepLog_id_ExecutionSte_202309260930.csv'
CSV_filename = '_SELECT_ExecutionLog_scriptname_ExecutionStepLog_id_ExecutionSte_202309281612.csv'
CSV_filename = '_SELECT_ExecutionLog_scriptname_ExecutionStepLog_id_ExecutionSte_202309300024.csv'
CSV_filename = '_SELECT_ExecutionLog_scriptname_ExecutionStepLog_id_ExecutionSte_202310231641.csv'

sql_extract = pd.read_csv("./data/execution/" + CSV_filename, low_memory=False)

sql_extract.rename(columns={'linenum': 'lineNum', 'executionid': 'executionLogsId', 'scriptname': 'scriptName', 'label': 'label', 'status': 'step_status', 'starttime': 'startTime', 'endtime': 'endTime', 'duration': 'duration', 'id': 'StepLogsId'}, inplace=True)

In [3]:
len(sql_extract)

4919696

In [4]:
rundate = '2023-10-23'
run_start_time = 11.0 # Time is UTC
run_finish_time = 23.0

In [5]:
df = sql_extract.copy()
df.rename(columns={'label':'StepLabel'}, inplace=True)
df = df.dropna(subset = ['scriptName']).copy()

In [6]:

df = df.dropna(subset=['startTime']).copy()
df = df.dropna(subset=['endTime']).copy()
df = df.sort_values(by=['startTime'], ascending=False).copy()
df = df.reset_index(drop=True)

In [7]:
mask = (df['duration'] == 'None') 
df['duration'].mask(mask,'0',inplace=True)
df['duration'] = pd.to_numeric(df['duration'])

In [8]:
df['duration_sec'] = df.duration/1000
df['duration_mins'] = df.duration/1000/60

In [9]:
#df['date'] = pd.to_datetime(df['startTime'], format='%Y-%m-%d', utc=True).dt.date
df['date'] = pd.to_datetime(df['startTime'], format='mixed', utc=True).dt.date
df['date'] = pd.to_datetime(df['date'])


In [10]:
df = df.query("date == @rundate").copy()  # Remember time is UTC

In [11]:
df['time'] = pd.to_datetime(df['startTime'], utc=True).dt.time

In [12]:
df['time_dec'] = df['time'].astype(str)
df['time_dec'] = df['time_dec'].str[:5]
df['time_dec'] = df['time_dec'].str.replace(':','.')
df['time_dec'] = df['time_dec'].astype(float)

### Add a column to indicate Runs of each test script
As there is no other field available via the API, this field is generated each time the lineNum == 1.  This assumes that there will always be a step 1 in every test case.
Before this can be done, first sort the whole dataframe

In [13]:
df.sort_values(by=['executionLogsId','StepLogsId'], inplace=True)
df = df.reset_index(drop=True)

In [14]:
# This is the new code to allocate a 'run' number and increment it every time the lineNum == 1.  This was generated by ChatGPT based on the code above :-)

df['run'] = (df['lineNum'] == 1).cumsum()

### Generate a field to indicate where test scripts have run to the last step of the script
This assumes that the last step is "End script"
First sort the dataframe by Runs
Then add a new text field that is a concatenation of all the step labels for each run
Finally, check which of those strings contain the text "End script"

In [15]:
runs_np = df.run.unique()
end_script_dict = {}
for i in runs_np:
    end_script_dict[i] = 'Script stopped'
step_error_dict = {}
for i in runs_np:
    step_error_dict[i] = 'No step errors'

In [16]:
df.sort_values(by=['run'], inplace=True)
df = df.reset_index(drop=True)

In [17]:
for idx in df.index:
    #if run == 10: break
    run = df['run'][idx]
    if df['StepLabel'][idx] == 'End script':
        end_script_dict[run] = 'Script completed'
    if df['step_status'][idx] == 'ERROR':
        step_error_dict[run] = 'One or more step errors'

In [18]:
end_script_df = pd.DataFrame.from_dict(end_script_dict,orient ='index',columns=['script_completion'])
end_script_df.reset_index(inplace=True)
end_script_df.rename(columns={'index':'run'}, inplace=True)

In [19]:
step_error_df = pd.DataFrame.from_dict(step_error_dict,orient ='index',columns=['step_error'])
step_error_df.reset_index(inplace=True)
step_error_df.rename(columns={'index':'run'}, inplace=True)

In [20]:
df = pd.merge(
    df,
    end_script_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [21]:
df = pd.merge(
    df,
    step_error_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [22]:
df.rename(columns={'script_status': 'EA_script_status','step_state': 'EA_step_state','step_status': 'EA_step_status','step_error': 'IPT_step_error','script_completion': 'IPT_script_completion'}, inplace=True)

In [23]:
df['KPI'] = 'Not mapped'

In [24]:
# K1
mask = (df['scriptName'].str.contains('Place a sales local order',case=False))
df['KPI'].mask(mask,'K01 Place a sales local order',inplace=True)

# K2
mask = (df['scriptName'].str.contains('Place a B2B local order',case=False))
df['KPI'].mask(mask,'K02 Place a B2B/EDI order and Send Confirmation',inplace=True)

# K3 Place Service Portal Order
# No test script

# K4
mask = (df['scriptName'].str.contains('Amend an existing sales order',case=False))
df['KPI'].mask(mask,'K04 Amend an existing sales order',inplace=True)

# K5
mask = (df['scriptName'].str.contains('Trade Returns Order',case=False))
df['KPI'].mask(mask,'K05 Place a Trade Returns Order',inplace=True)

# K6
mask = (df['scriptName'].str.contains('Submit a B2B Remittance',case=False))
df['KPI'].mask(mask,'K06 B2B Remittance',inplace=True)

# K7 B2B Remittance Adjustment
# No test script

#K8
mask = (df['scriptName'].str.contains('Receipted payment processing',case=False))
df['KPI'].mask(mask,'K08 Receipted payment processing SA Only at the moment',inplace=True)

# K9 aDSD Batch Job posting invoice for HHD billing document 
# No test script

# K10
mask = (df['scriptName'].str.contains('Credit status check',case=False))
df['KPI'].mask(mask,'K10 Credit status check',inplace=True)

# K11
mask = (df['scriptName'].str.contains('Send Load to Roadnet',case=False))
df['KPI'].mask(mask,'K11 Send Load to Roadnet load for Planning',inplace=True)

# K12
mask = (df['scriptName'].str.contains('Receive load from Roadnet',case=False))
df['KPI'].mask(mask,'K12 Receive load from Roadnet into D365',inplace=True)

# K13b
mask = (df['scriptName'].str.contains('Release to warehouse',case=False) & df['scriptName'].str.contains('Roadnet loads',case=False))
df['KPI'].mask(mask,'K13b Release to Warehouse & Complete Picking Work (Roadnet loads)',inplace=True)
# Release to Warehouse (Roadnet loads) & Complete Picking Work & Process OOS

# K13
mask = (df['scriptName'].str.contains('Release to Warehouse',case=False) & df['scriptName'].str.contains('manual',case=False))
df['KPI'].mask(mask,'K13 Release to Warehouse (manually planned loads)',inplace=True)

# K14 Complete Picking Work & Process OOS 
# Not mapped??


# K14b
mask = (df['scriptName'].str.contains('Complete Picking Work',case=False) & df['scriptName'].str.contains('manual loads',case=False))
df['KPI'].mask(mask,'K14b Complete Picking Work & Process OOS (manually planned loads)',inplace=True)

# K15
mask = (df['scriptName'].str.contains('aDSD Load confirmation',case=False))
df['KPI'].mask(mask,'K15 aDSD Load confirmation',inplace=True)

# K16
mask = (df['scriptName'].str.contains('Load upload',case=False) & df['scriptName'].str.contains('Settlement',case=False))
df['KPI'].mask(mask,'K16 Load upload & Settlement',inplace=True)

# K17
mask = (df['scriptName'].str.contains('Create a cost estimate',case=False))
df['KPI'].mask(mask,'K17 Create a cost estimate for all standard costed procured materials',inplace=True)

# K18
mask = (df['scriptName'].str.contains('Imported Statistical',case=False))
df['KPI'].mask(mask,'K18 Import Actual Statistical entries, into Cost Accounting',inplace=True)

# K19
mask = (df['scriptName'].str.contains('Distribute the range of items pending prices',case=False))
df['KPI'].mask(mask,'K19 Distribute the range of items pending prices to another site',inplace=True)

# K20
mask = ((df['scriptName'].str.contains('Cost Rollup',case=False) & df['scriptName'].str.contains('Cost Allocation',case=False)) | df['scriptName'].str.contains('Maintain cost distribution table',case=False))
df['KPI'].mask(mask,'K20 Run Cost Rollup and Cost Allocation Policies',inplace=True)

# K21
mask = (df['scriptName'].str.contains('Place SFA order',case=False))
df['KPI'].mask(mask,'K21 Place SFA order and Send Confirmation',inplace=True)

In [25]:
df.sort_values(by=['KPI', 'scriptName', 'executionLogsId', 'run', 'lineNum'], inplace=True)
df = df.reset_index(drop=True)

In [26]:
df = df.dropna(subset=['StepLabel']).copy()

### Filter the dataset for output on a specific test cycle

In [27]:
df1 = df.query("date == @rundate and time_dec > @run_start_time and time_dec < @run_finish_time").copy()  # Remember time is UTC

In [28]:
df1 = df.reset_index(drop=True).copy()

In [29]:
successful_runs = df1[df1['IPT_script_completion'] == 'Script completed'].groupby('KPI').agg({'run': 'nunique'}).reset_index()


In [30]:
incomplete_runs = df1[df1['IPT_script_completion'] != 'Script completed'].groupby('KPI').agg({'run': 'nunique'}).reset_index()
incomplete_runs.rename(columns={'run': 'Incomplete Runs', 'KPI': 'KPI Process'}, inplace=True)

### Mark those Test Script Steps that must be included in the time measurement

In [31]:
df1['KPI_no'] = ''

In [32]:
df1['Include_in_measure'] = False

In [33]:
kpi_numbers = ['K11', 'K12', 'K13', 'K14', 'K15', 'K16', 'K17', 'K18', 'K19', 'K20']

In [34]:
for idx in tqdm(df1.index):
    label = df1['StepLabel'][idx]
    prefix = label[0:3]
    if prefix in kpi_numbers:
        if label[3] == 'b':
            df1.at[idx, 'KPI_no'] = prefix + 'b'
        else:
            df1.at[idx, 'KPI_no'] = prefix  
        df1.at[idx, 'Include_in_measure'] = True

 42%|████▏     | 41458/97807 [00:00<00:00, 211360.75it/s]

100%|██████████| 97807/97807 [00:00<00:00, 167383.41it/s]


In [35]:
filename = 'data/execution/' + rundate +  '_results_filtered.csv'
df1.to_csv(filename,index=False)

In [36]:
#df2 = df1.query("Include_in_measure == True and EA_step_status != 'WARNING' and EA_step_status != 'ERROR'").copy()
df2 = df1.query("Include_in_measure == True and EA_step_status != 'ERROR'").copy()

In [37]:
df2 = df2.drop_duplicates(keep='first')
df2 = df2.reset_index(drop=True)

In [38]:
# 90th Percentile
def q90(x):
    return x.quantile(0.9)


kpi_results = df2.groupby(['KPI_no']).agg({'KPI': 'first', 'duration_sec': ['mean', 'max', q90, 'std'], 'run': 'count'}).reset_index()

# Flatten multi-level columns
kpi_results.columns = ['_'.join(col) for col in kpi_results.columns]
kpi_results = kpi_results.reset_index(drop=True)

In [39]:
kpi_results.rename(columns={'KPI_no_': 'KPI_no', 'KPI_first':'KPI Process', 'duration_sec_mean': 'Average', 'duration_sec_max': 'Max', 'duration_sec_q90': '90th_%', 'duration_sec_std': 'SD', 'run_count': 'Successful Runs'}, inplace=True)

In [40]:
kpi_results = pd.merge(
    kpi_results,
    incomplete_runs,
    how="left",
    on=None,
    left_on='KPI Process',
    right_on='KPI Process',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

kpi_results['Incomplete Runs'] = kpi_results['Incomplete Runs'].fillna(0)

In [41]:
kpi_results = kpi_results.round(decimals=1)

In [42]:
kpi_results

,KPI_no,KPI Process,Average,Max,90th_%,SD,Successful Runs,Incomplete Runs
0,K11,K11 Send Load to Roadnet load for Planning,3.1,10.3,4.6,1.2,132,12
1,K12,K12 Receive load from Roadnet into D365,211.5,493.2,487.4,185.7,8,11
2,K13,K13 Release to Warehouse (manually planned loads),16.3,65.9,33.7,13.6,39,12
3,K14,K13 Release to Warehouse (manually planned loads),15.9,154.8,18.5,29.8,39,12
4,K13b,K13b Release to Warehouse & Complete Picking W...,23.9,1818.0,35.2,83.9,472,47
5,K14b,K13b Release to Warehouse & Complete Picking W...,9.1,422.0,19.1,21.0,1449,47
6,K15,K15 aDSD Load confirmation,83.7,612.2,282.7,174.5,128,24


In [43]:
filename = 'data/execution/' + rundate +  '_kpi_results.xlsx'
kpi_results.to_excel(filename,index=False)

In [44]:
raise SystemExit("Stop right here!")

SystemExit: Stop right here!

/home/wikus/code/ipt3/env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
